In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

importing gdp data set

In [ ]:
gdp_df = pd.read_csv('../data/gdp_percapita.csv').drop(columns = {'Value Footnotes'})

In [ ]:
gdp_df = gdp_df.rename(columns = {'Country or Area':'Country', 'Value':'GDP_Per_Capita'})

In [ ]:
gdp_df.info()

In [ ]:
gdp_df

importing and cleaning internet dataset

In [ ]:
internet_df_full = pd.read_csv('../data/internet_use.csv', usecols = ['Country or Area', 'Year', 'Value', 'Value Footnotes'])
internet_df = internet_df_full[0:4495]
internet_df = internet_df.rename(columns = {'Country or Area':'Country', 'Value':'Internet_Users_Pct'}).drop(columns = ['Value Footnotes'])
internet_footnoteIDs_df = internet_df_full[4496:4685].drop(columns = ['Value', 'Value Footnotes']).rename(columns = {'Country or Area':'footnoteSegID', 'Year':'Footnote'}).reset_index(drop=True)

In [ ]:
internet_df['Year'] = internet_df['Year'].astype(int)
internet_df['Internet_Users_Pct'] = internet_df['Internet_Users_Pct'].astype(float)

In [ ]:
internet_df.info()

In [ ]:
internet_df

### 20. Merge the two DataFrames to one. Do this in a way that keeps **all rows** from each of the two DataFrames. Call the new DataFrame `gdp_and_internet_use`. Look at the first and last few rows to confirm that it merged correctly.

In [ ]:
gdp_and_internet_use = pd.merge(gdp_df, internet_df,
                                on = ('Country', 'Year'),
                                how = 'inner')

In [ ]:
gdp_and_internet_use.info()

In [ ]:
gdp_and_internet_use

### 21. Find the three countries with the highest internet users percentage in 2014. Use a seaborn FacetGrid (https://seaborn.pydata.org/generated/seaborn.FacetGrid.html) to compare how the GDP per capita has changed over time for these three countries. What do you notice?

In [ ]:
year_2014 = gdp_and_internet_use[gdp_and_internet_use['Year'] == 2014]

In [ ]:
year_2014 = year_2014.sort_values(by = 'Internet_Users_Pct', ascending = False).head(3)

In [ ]:
year_2014

In [ ]:
topInternet_countries2014 = list(year_2014['Country'])
topInternet_countries2014

Iceland 98.16% ; Bermuda 96.80% ; Norwway 96.30%

creating a dataframe with only the top three countries in 2014

In [ ]:
top3_2014 = gdp_and_internet_use[(gdp_and_internet_use['Country'] == 'Iceland') | (gdp_and_internet_use['Country'] == 'Bermuda') | (gdp_and_internet_use['Country'] == 'Norway')]
top3_2014

##### Note
g = sns.FacetGrid and g.map must be in the same cell to properly work

In [ ]:
g = sns.FacetGrid(top3_2014, col='Country')
g.map(sns.scatterplot, 'Year', 'GDP_Per_Capita')
ax1, ax2, ax3 = g.axes[0]
ax1.axvline(2014, color = 'red')
ax2.axvline(2014, color = 'red')
ax3.axvline(2014, color = 'red');

GDP was not maximum in the year 2014 for any of the three countries. GDP may have been relatively low for all countries during 2014 (requires further investigation)

### 22. Subset `gdp_and_internet_use` to just the year 2014. Save this as a new dataframe named `gdp_and_internet_use_2014`.

In [ ]:
gdp_and_internet_use_2014 = gdp_and_internet_use[gdp_and_internet_use.Year == 2014]
gdp_and_internet_use_2014

### 23. Create a plot which compares Internet Users Percentage and GDP per Capita for the year 2014. What do you notice from this plot? If you see any unusual points, investigate them.

normalizing the gdp by dividing 1,000

In [ ]:
gdp_and_internet_2014_normal = gdp_and_internet_use_2014.copy(deep = True)
gdp_and_internet_2014_normal['GDP_Per_Capita'] = gdp_and_internet_use_2014['GDP_Per_Capita'].div(1000)
gdp_and_internet_2014_normal

In [ ]:
sum_2014 = gdp_and_internet_2014_normal.groupby(by = ('Year')).sum()
mean_2014 = gdp_and_internet_2014_normal.groupby(by = ('Year')).mean()
max_2014 = gdp_and_internet_2014_normal.groupby(by = ('Year')).max()
min_2014 = gdp_and_internet_2014_normal.groupby(by = ('Year')).min()

In [ ]:
sns.barplot(data = sum_2014)

In [ ]:
sns.barplot(data = mean_2014)

In [ ]:
sns.barplot(data = max_2014)

In [ ]:
sns.barplot(data = min_2014)

bar chart information seems inutile; scatterplot may provide better insight

In [ ]:
sns.regplot(data=gdp_and_internet_use_2014, x='Internet_Users_Pct', y='GDP_Per_Capita');

There is a positive correlation with GDP and Internet Users

### 24. **Stretch Question:** Use the `qcut` function from pandas (https://pandas.pydata.org/docs/reference/api/pandas.qcut.html) to divide countries in `gdp_per_capita_2014` into three groups based on their GDP per capita values. Label these groups as "Low", "Medium", and "High". Put these labels in a new column, named "GDP_group".

In [ ]:
gdp_and_internet_use_2014['GDP_group'] = pd.qcut(gdp_and_internet_use['GDP_Per_Capita'], 3, labels = ['Low', 'Medium', 'High'])
gdp_and_internet_use_2014

### 25. **Stretch Question:** How does the median internet users percentage compare for the three gdp groups?

In [ ]:
gdp_and_internet_use_2014.groupby(by=('GDP_group'))['Internet_Users_Pct'].mean()